In [93]:
import os
import csv
import time
import psycopg2
import sqlalchemy
import pandas as pd
from faker import Faker
fake = Faker()
print(fake.name())

Cynthia Foster


In [95]:
os.environ['DB_NAME'] = 'api_db'
os.environ['DB_USER'] = 'user_chandu'
os.environ['DB_PASSWORD'] = '0009'
os.environ['DB_HOST'] = 'localhost'
os.environ['DB_PORT'] = '5432'

In [97]:
def generate_csv(filename='sales_data.csv', rows=1000000):
    fake = Faker()
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['customer_name', 'product', 'quantity', 'price', 'order_date'])
        
        for _ in range(rows):
            writer.writerow([
                fake.name(),
                fake.word(),
                fake.random_int(min=1, max=100),
                round(fake.random_number(digits=5) / 100, 2),
                fake.date_between(start_date='-2y', end_date='today')
            ])

generate_csv('sales_data.csv', 1000000)


In [98]:
def get_engine():
    db_url = (
        f"postgresql+psycopg2://{os.environ['DB_USER']}:{os.environ['DB_PASSWORD']}"
        f"@{os.environ['DB_HOST']}:{os.environ['DB_PORT']}/{os.environ['DB_NAME']}"
    )
    return sqlalchemy.create_engine(db_url)

engine = get_engine()


In [99]:
df = pd.read_csv('sales_data.csv')
df.head()

,customer_name,product,quantity,price,order_date
0,Eric Stevens,begin,79,7.97,2024-10-28
1,Jennifer Maxwell,party,49,437.75,2024-04-02
2,Diana Schneider,treatment,56,891.99,2025-03-27
3,Dawn Brown,movie,10,236.77,2025-05-01
4,Dr. Jennifer Rodriguez PhD,skill,6,806.23,2025-04-23


In [101]:
def batch_insert(df):
    start =time.time()
    with engine.begin() as conn:
        conn.exectue(
            """
            INSERT INTO sales_data (customer_name, product, quantity, price, order_date)
            VALUES(%s, %s, %s, %s, %s)
            """,
            [
                (
                    row["customer_name"],
                    row["product"],
                    int(row["quantity"]),
                    float(row[" price"]),
                    row["order_date"]
                )
                for _, row in df.iterrows()
            ]
        )
    print("Batch Insert Time:", time.time() -start)

In [102]:
def batch_insert(df, batch_size=10000):
    engine = get_engine()
    start = time.time()
    df.to_sql('sales_data', engine, index=False , if_exists='append', method='multi', chunksize=batch_size)
    print("Batch Insert Time:", round(time.time() -start,2),"seconds")

batch_insert(df.head(500000))


Batch Insert Time: 19.34 seconds


In [103]:
def copy_insert(filename='sales_data.csv'):
    conn = psycopg2.connect(
        dbname=os.environ['DB_NAME'],
        user=os.environ['DB_USER'],
        password=os.environ['DB_PASSWORD'],
        host=os.environ['DB_HOST'],
        port=os.environ['DB_PORT']
    )
    cur =conn.cursor()
    start = time.time()
    
    with open(filename, 'r') as f:
        next(f)
        cur.copy_expert("""
            COPY sales_data(customer_name, product, quantity, price, order_date)
            FROM STDIN WITH CSV
        """, f)
    conn.commit()
    cur.close()
    conn.close()
    print("Copy Insert Time:" ,time.time() - start)

copy_insert('sales_data.csv')

Copy Insert Time: 0.7276678085327148


In [116]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM sales_data LIMIT 5"))
    for row in result:
        print(row)

('Wayne Evans', 'vote', 11, 836.57, '2024-05-04')
('Alan Medina', 'reality', 91, 173.22, '2024-12-21')
('Kelly Torres', 'cell', 68, 301.16, '2024-02-10')
('James Perry', 'thus', 5, 208.89, '2024-10-29')
('Michael Brewer', 'eight', 37, 108.4, '2024-12-25')
